In [1]:
import pytest
from matsciml.datasets import lips
from matsciml.datasets.lips import LiPSDataset, lips_devset
from matsciml.datasets import transforms
import pytorch_lightning as pl
from matsciml.models.pyg import EGNN
from matsciml.models.base import ScalarRegressionTask
from matsciml.lightning.data_utils import MatSciMLDataModule
from matsciml.datasets.transforms import DistancesTransform

from matsciml.models.pyg.mace.modules.blocks import *
from matsciml.models.pyg.mace.modules.models import ScaleShiftMACE

import matsciml
import e3nn

c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from matsciml.datasets.transforms import PointCloudToGraphTransform
# SchNet uses RBFs, and expects edge features corresponding to atom-atom distances
dm = MatSciMLDataModule.from_devset(
    "LiPSDataset",dset_kwargs={"transforms":[PointCloudToGraphTransform("pyg", cutoff_dist=5.0)]}
)
# run a quick training loop
# trainer = pl.Trainer(fast_dev_run=1000)
# trainer.fit(task, datamodule=dm)


In [5]:
model_config = dict(
        r_max=5.0,
        num_bessel=8,
        num_polynomial_cutoff=5,
        max_ell=3,
        interaction_cls= RealAgnosticResidualInteractionBlock ,
        num_interactions=2,
        num_elements=3,
        hidden_irreps=e3nn.o3.Irreps('16x0e+16x1o+16x2e'),
        atomic_energies=torch.Tensor([-13.663181292231226, -1029.2809654211628, -2042.0330099956639]),
        avg_num_neighbors=14.38,
        atomic_numbers=(1,6,8),
        correlation=3,
        gate=torch.nn.functional.silu,
        interaction_cls_first=RealAgnosticResidualInteractionBlock,
        MLP_irreps=e3nn.o3.Irreps('16x0e'),
        atomic_inter_scale=1.0,
        atomic_inter_shift=0.0
    )

In [6]:
task = ScalarRegressionTask(
    encoder_class=ScaleShiftMACE,
    # kwargs to be passed into the creation of SchNet model
    encoder_kwargs=model_config,
    # which keys to use as targets
    task_keys=["energy"],
    output_kwargs={
      "block_type": "IrrepOutputBlock",
      "input_dim": "0e", 
      "hidden_dim": "30x0e + 10x1e", 
      "output_dim": "0e",
      "activation": ["torch.nn.SiLU", None]
    }
)

c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\torch\jit\_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
f:\Vaibhav\M3RG\matsciml\matsciml\models\pyg\mace\modules\blocks.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(atomic_energies, dtype=torch.get_default_dtype()),


In [8]:
trainer = pl.Trainer(fast_dev_run=1000)
trainer.fit(task, datamodule=dm)

GPU available: True (cuda), used: False


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
Running in `fast_dev_run` mode: will run the requested loop using 1000 batch(es). Logging and checkpointing is suppressed.

  | Name         | Type           | Params
------------------------------------------------
0 | encoder      | ScaleShiftMACE | 58.8 K
1 | loss_func    | MSELoss        | 0     
2 | output_heads | ModuleDict     | 1.1 K 
------------------------------------------------
59.9 K    Trainable params
0         Non-trainable params
59.9 K    Total params
0.240     Total estimated model params size (MB)
c:\Users\DIKSHIKA\miniconda3\envs\matsciml\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:

Epoch 0:   0%|          | 0/50 [00:00<?, ?it/s] 

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "<eval_with_key>.232", line 25, in forward
    new_zeros = reshape_1.new_zeros((getitem_1, 30));  reshape_1 = getitem_1 = None
    cat = torch.cat([add_1, new_zeros], dim = -1);  add_1 = new_zeros = None
    reshape_7 = cat.reshape(add);  cat = add = None
                ~~~~~~~~~~~ <--- HERE
    return reshape_7
RuntimeError: shape '[60]' is invalid for input of size 480
